In [11]:
import pickle as pkl
import torch

components_list = [
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_0shot_SFTCodeS-15b',
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_1Of0-5shot',
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_2Of0-5shot',
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_3Of0-5shot',
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_4Of0-5shot',
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_5Of0-5shot'
]
prob_postfix_list = [
    '_joint0sh_beams_log_prob.pkl',
    '_joint1sh1_beams_log_prob.pkl',
    '_joint1sh2_beams_log_prob.pkl',
    '_joint1sh3_beams_log_prob.pkl',
    '_joint1sh4_beams_log_prob.pkl',
    '_joint1sh5_beams_log_prob.pkl'
]
output_postfix = '_joint6comp_beams_log_prob.pkl'

for comp_candidates in components_list:
    print(comp_candidates)
    log_sum = None
    for postfix in prob_postfix_list:
        filename= comp_candidates + postfix
        with open(filename , 'rb' ) as f:
            prob_list = pkl.load(f)
            # prob_list = torch.load(f, map_location=torch.device('cpu'), weights_only=False)
        print( prob_list[0:4] )
        prob_tensor = torch.stack(prob_list)
        if log_sum is None:
            log_sum = prob_tensor
        else:
            # print(f'selection_score_with_other_response:{selection_score_with_other_response} --> {selection_score_with_other_response.dtype}')
            # print(f'selection_score:{selection_score} --> {selection_score.dtype}')
            alpha = torch.maximum( prob_tensor , log_sum )
            beta = torch.minimum( prob_tensor , log_sum )
            log_sum = alpha + torch.log1p(torch.exp(beta-alpha))
    log_sum_list = log_sum.tolist()
    with open(comp_candidates+output_postfix , 'wb') as f:
        pkl.dump(log_sum_list, f)
        

./outputs/raw/output_CodeS_candid16_bird_wEvidence_0shot_SFTCodeS-15b
[tensor(-1.8258), tensor(-1.8306), tensor(-1.7763), tensor(-1.8396)]
[tensor(-1.7356), tensor(-1.7406), tensor(-1.6803), tensor(-1.7404)]
[tensor(-1.6017), tensor(-1.5796), tensor(-1.4929), tensor(-1.5882)]
[tensor(-1.6526), tensor(-1.6483), tensor(-1.5728), tensor(-1.6491)]
[tensor(-1.7751), tensor(-1.7806), tensor(-1.6943), tensor(-1.7639)]
[tensor(-1.6166), tensor(-1.6155), tensor(-1.5652), tensor(-1.5998)]
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_1Of0-5shot
[tensor(-1.8445), tensor(-1.8258), tensor(-1.8193), tensor(-1.8004)]
[tensor(-1.7313), tensor(-1.7363), tensor(-1.7454), tensor(-1.7088)]
[tensor(-1.5796), tensor(-1.6017), tensor(-1.6030), tensor(-1.5470)]
[tensor(-1.6731), tensor(-1.6526), tensor(-1.6689), tensor(-1.5980)]
[tensor(-1.8124), tensor(-1.7751), tensor(-1.7888), tensor(-1.7421)]
[tensor(-1.6537), tensor(-1.6166), tensor(-1.6875), tensor(-1.5531)]
./outputs/raw/output_

In [12]:
import math
num_beam = 16 # We should not forget to set this parameter correctly!!
new_candid_prob_list = []
new_output_prefix = '_a1b1Joint6comp_beams_log_prob.pkl'
for beam_index in range(0,1534*num_beam,num_beam):
    candid_prob_list = []
    for idx , comp_candidates in enumerate(components_list):
        
        if idx <= len(new_candid_prob_list):
            new_candid_prob_list.append([])
            
        filename= comp_candidates + output_postfix
        with open(filename , 'rb' ) as f:
            prob_list = pkl.load(f)
        candid_prob_list.extend(prob_list[beam_index:beam_index+num_beam])
        
    candid_prob_list_min = min(candid_prob_list)
    candid_prob_list_max = max(candid_prob_list)
    candid_prob_list_scaled = [(x - candid_prob_list_min) / (candid_prob_list_max - candid_prob_list_min) for x in candid_prob_list]
    
    # Step 2: Apply exponentiation to amplify differences
    # alpha = 1  # You can experiment with different values #It kinda give the original value
    # beta = .087 #It kinda give the original value
    # gama = 0.275 #It kinda give the original value
    alpha = 1  
    beta = 1 
    gama = 0 
    p_transformed = [beta * x ** alpha + gama for x in candid_prob_list_scaled]
    for i in range(0, len(p_transformed), num_beam):
        new_candid_prob_list[ math.ceil(i/num_beam) ].extend(p_transformed[i:i+num_beam])

for prob_list, comp_candidates in zip(new_candid_prob_list, components_list):
    with open(comp_candidates+new_output_prefix , 'wb') as f:
        pkl.dump(prob_list, f)



In [10]:
#0:1 | 2:1 | 4:1 | 5:2 | 7:3
for comp_candidates in components_list:
    filename= comp_candidates + output_postfix# '.pkl'
    with open(filename , 'rb' ) as f:
        prob_list = pkl.load(f)
    print(comp_candidates)
    print(len(prob_list))
    # for i in prob_list[0:4]:
    #     print(i)

./outputs/raw/output_CodeS_candid16_bird_wEvidence_0shot_SFTCodeS-7b
24544
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-7b_1Of0-5shot
24544
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-7b_2Of0-5shot
24544
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-7b_3Of0-5shot
24544
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-7b_4Of0-5shot
24544
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-7b_5Of0-5shot
24544


In [71]:
torch.exp(torch.tensor(-0.9748774810717132))
math.log(8)

2.0794415416798357